__Chapter 16 - Modeling Sequential Data Using Recurrent Neural Networks__

1. [Introducing sequential data](#Introducing-sequential-data)
    1. [Modeling sequential data – order matters](#Modeling-sequential-data–order-matters)
    1. [Representing sequences](#Representing-sequences)
    1. [The different categories of sequence modeling](#The-different-categories-of-sequence-modeling)
1. [RNNs for modeling sequences](#RNNs-for-modeling-sequences)
    1. [Understanding the structure and flow of an RNN](#Understanding-the-structure-and-flow-of-an-RNN)
    1. [Computing activations in an RNN](#Computing-activations-in-an-RNN)
    1. [The challenges of learning long-range interactions](#The-challenges-of-learning-long-range-interactions)
1. [Implementing a multilayer RNN for sequence modeling in TensorFlow](#Implementing-a-multilayer-RNN-for-sequence-modeling-in-TensorFlow)
    1. [Project one - performing sentiment analysis of IMDb movie reviews using multilayer RNNs](#performing-sentiment-analysis-of-IMDb-movie-reviews-using-multilayer-RNNs)
        1. [Preparing the data](#Preparing-the-data)
        1. [Embedding](#Embedding)
        1. [Building an RNN model](#Building-an-RNN-model)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)



In [1]:
# Standard libary and settings
import os
import sys
import importlib
import itertools
import warnings; warnings.simplefilter('ignore')
dataPath = os.path.abspath(os.path.join('../../Data'))
modulePath = os.path.abspath(os.path.join('../../CustomModules'))
sys.path.append(modulePath) if modulePath not in sys.path else None
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))


# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.6f}'.format


# Modeling extensions
import sklearn.base as base
import sklearn.cluster as cluster
import sklearn.datasets as datasets
import sklearn.decomposition as decomposition
import sklearn.discriminant_analysis as discriminant_analysis
import sklearn.ensemble as ensemble
import sklearn.feature_extraction as feature_extraction
import sklearn.feature_selection as feature_selection
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
import sklearn.neighbors as neighbors
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
import sklearn.svm as svm
import sklearn.tree as tree
import sklearn.utils as utils


# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt


# Custom extensions and settings
from quickplot import qp, qpUtil, qpStyle
from mlTools import powerGridSearch
sns.set(rc = qpStyle.rcGrey)


# Magic functions
%matplotlib inline


<a id = 'Introducing-sequential-data'></a>

# Introducing sequential data

This chapter explores the unique properties of sequences compared to other kinds of data. We will explore how we can represent sequential data and the various models for analyzing sequential data.

<a id = 'Modeling-sequential-data–order-matters'></a>

## Modeling sequential data – order matters

One major unique aspect of sequential data is that the elements appear in a certain order and are not independent of each other. The contrasts with data and algorithms that we have dealth with up to this point, in that previous models assume that the data is independent and identically distributed (IID). But with sequential data, by definition, order matters. This is not necessarily a problem, and in fact, the order can yield meaningful information. We just need a different approach and different tools.

<a id = 'Representing sequences'></a>

## Representing sequences

In this chaper, sequences will be represented as $\big(x^1, x^2,...,x^T\big)$, where the superscript indices indicate the order of the instances, and the length of the sequence is $T$. For example, in time-series data, sample $\textbf{x}^T$ belongs to a particular time $t$. Further, if the data is labeled, the labels also follow a form where order matters: $\big(y^1, y^2,...,y^T\big)$.

The MLP and CNN models built in the last few chapters are not capable of handling the order of the input simples. Recurrent neural networks (RNNs) are designed to model sequences that remember past information and process new events in light of that history.

<a id = 'The-different-categories-of-sequence-modeling'></a>

## The different categories of sequence modeling

Sequence modeling can be applied to, among other things, language translateion, image cpationing, and text generation. Sequential data comes in many forms, and the nature of the input and output data determines the type. If neither the input nor the output data is sequenced, then this is simply a standard dataset, any of the methods covered in previous chapter may be used (depending on the problem).

If either the input or output data is sequenced, then it can be identified by one of these three categories:

- Many-to-one: The input data is sequenced, but the output is a vector of a fixed size, not a sequence. For example, sentiment analysis takes text data as an input and outputs a class label.
- One-to-many: The input data is in a standard format (not sequenced) but the output is a seuqnce. For example, in image captioning, the input is an image and the ouput is an English phrase.
- Many-to-many: Both the input and output arrays are sequences. This category can be sub-divided into subcategories based on whether the input or output is synchronized or not. An example of synchronized many-to-many is video classification, where each from in a video is labeled. An example of delayed many-to-many is language translation, i.e. an English sentence is translated by a machine into its equivalent in German

<a id = 'RNNs-for-modeling-sequences'></a>

# RNNs for modeling sequences

This sections describes the foundations of RNNs, including typical structure, dat flow, neuron activation, and typical challenges.

<a id = 'Understanding-the-structure-and-flow-of-an-RNN'></a>

## Understanding the structure and flow of an RNN

In a feedforward network, information flows from the input layer, to the hidden layer(s), then to the output layer. In an RNN, the hidden layer gets its input from both the input layer and the hidden layer from the previous time step. This flow of information in adjacent time steps in the hidden layer allows the network to use its 'memory of past events'. This can be envisioned as a loop, which in graph notation is referred to as a recurrent edge. This can be visualized as:

$$
\textbf{x}^t \rightarrow \textbf{h}^t \rightarrow \textbf{y}^t 
$$

where $\textbf{x}^t$ is the input data at the $t$ point in the sequence, $\textbf{h}^t$ is the hidden layer at point $t$, and $\textbf{y}^t$ at point $t$. This can be unfolded to reveal how other data points observed at adjacent time steps are structured relative to point $t$:

$$
\textbf{x}^{t-1} \rightarrow \textbf{h}^{t-1} \rightarrow \textbf{y}^{t-1}
\\
\downarrow
\\
\textbf{x}^t \rightarrow \textbf{h}^t \rightarrow \textbf{y}^t 
\\
\downarrow
\\
    \textbf{x}^{t+1} \rightarrow \textbf{h}^{t+1} \rightarrow \textbf{y}^{t+1}
$$

RNNs can have multiple hidden layers as well.

In a standard neural network, each hidden unit only receives one input - the net input associated with the input layer. RNNs, conversely, neurons in the hidden layer receive two distinct inputs - the net input from the input layer and the net input of the same hidden layer neuron from the previous time step $t-1$. At $t=0$, the first time step, the hidden units are initialized to zeros, or small random numbers. Then for $t>0$, the hidden units get input from the data point at the current time $\textbf{x}^t$ and the previous values of the hidden units at $t-1$, $\textbf{h}^{t-1}$

<a id = 'Computing-activations-in-an-RNN'></a>

## Computing activations in an RNN

Each directed edge (connection between boxes) of an RNN is associated with a weight matrix, and these weights do not depend on time $t$. These weights are shared across the time axis. The different weight matrices in a single layer RNN are:

$$
\textbf{W}_{xh}: \mbox{the weight matrix between the input} \ \textbf{x}^t \mbox{and the hidden layer } \ \textbf{h}
\\
\textbf{W}_{hh}: \mbox{the weight matrix associated with the recurrent edge}
\\
\textbf{W}_{hy}: \mbox{the weight matrix between the hidden layer and the output layer}
$$

Again, these weight matrices apply to the current point in the sequence $t$, as well as to $t-1$ and $t+1$.

The activations are computed similar to how this is handled in feed forward networks. For example, in the hidden layer, the net input $\textbf{z}_h$ is computed through a linear combination determined by summing the multiplications of the weight matrices with the corresponding vectors, and adding the bias unit:

$$
\textbf{z}_h^t = \textbf{W}_{xh}\textbf{x}^t + \textbf{W}_{hh}\textbf{h}^{t-1} + \textbf{b}_h
$$

Then the activations of the hidden units at the time step $t$ are calculated using:

$$
\textbf{h}^t = \phi_h\big(\textbf{z}_h^t\big) = \phi_h\big(\textbf{W}_{xh}\textbf{x}^t + \textbf{W}_{hh}\textbf{h}^{t-1} + \textbf{b}_h\big)
$$

where $\phi_h(\cdot)$ is the activation function.

Once the activations of the hidden units at the current time step are calculated, the activations of the output units are calculated by:
$$
\textbf{y}^t = \phi_y\big(\textbf{W}_{hy}\textbf{h}^t + \textbf{b}_y\big)
$$


<a id = 'The-challenges-of-learning-long-range-interactions'></a>

## The challenges of learning long-range interactions

Backpropagation through time (BPTT) is the process for optimizing the weights in an RNN. The basic idea is that the overall loss $L$ is the sum of all loss functions calculated at times $t$ = 1 to $t$ = $T$. 

$$
L = \sum^T_{t=1}L^t
$$

The loss at time 1:$t$ is dependent on the hidden units at all time steps that were evaluated before 1:$t$, so the gradient is calculated as follows:

$$
\frac{\partial L^t}{\partial\textbf{W}_{hh}} = \frac{\partial L^t}{\partial\textbf{y}^{t}} \times \frac{\partial \textbf{y}^t}{\partial\textbf{h}^{t}} \times \Bigg(\sum^t_{k=1}\frac{\partial \textbf{h}^t}{\partial\textbf{h}^{k}} \times \frac{\partial \textbf{h}^k}{\partial\textbf{h}_{hh}}\Bigg)
$$

In this formula $\frac{\partial \textbf{h}^t}{\partial\textbf{h}^{k}}$ is computed as multiplication of adjacent time steps:

$$
\frac{\partial \textbf{h}^t}{\partial\textbf{h}^{k}} = \prod^t_{i=k+1}\frac{\partial \textbf{h}^i}{\partial\textbf{h}^{i-1}}
$$

Calculation of the term $\frac{\partial \textbf{h}^t}{\partial\textbf{h}^{k}}$ introduces a few challenges. Namely, the so-called vanishing/exploding gradient. This term has $t-k$ multiplications, so multiply the $w$ weight a total of $t - k$ times results in a factor $w^{t-k}$. As a result, if $\lvert w\rvert$ < 1, this factor becomes very small when $t-k$ is large. On the other hand, if $\lvert w\rvert$ > 1, then $w^{t-k}$ becomes very large when $t-k$ is large. This means that we prefer $w$ to be equal to 1.

There are two solutions to this problem:

- Truncated backpropagation through time (TBPTT): clips the gradients above a given threshold. This solves exploding gradient issues, but the truncation limits the number of steps the gradient can effectively flow back and update weights properly
- Long short-term memory (LSTM): introduced to overcome the vanishing gradient problem. More successful in modeling long-range sequences than TBPTT. 


<a id = 'Implementing-a-multilayer-RNN-for-sequence-modeling-in-TensorFlow'></a>

# Implementing a multilayer RNN for sequence modeling in TensorFlow

The rest of this notebook will explore RNN implementations to address two common tasks, sentiment analysis and language models.

<a id = 'performing-sentiment-analysis-of-IMDb-movie-reviews-using-multilayer-RNNs'></a>

## Project one – performing sentiment analysis of IMDb movie reviews using multilayer RNNs

In chapter 8, we implemented a model to determine the sentiment of movie reiews on IMDb. This project will leverage an RNN model to do the same task. This is an example of a many-to-one problem, where we are given a document of text and need to return a single label

<a id = 'Preparing-the-data'></a>

### Preparing the data

This dataset contains two columns, one with the movie reviews, and another with the sentiment label of 0 or 1. The text component of these movie reviews are sequences of words, so we want to build an RNN to process the words in sequence and then classify the entire sequence to the 0 or 1 class.

To make this dataset ready for the neural network, it needs to be encoded into numeric values. First, we need to find the unique words in the entire dataset. This is not the same as preparing a bag-of-words model, as we are only interested in the set of unique words, and we don't need the counts necessarily. Second we create a mapping by way of a dictionary where we pair each unique word with a unique integer number. This will convert the entire text into a list of numbers.

In [9]:
# Load ImdbReviews dataset

import pyprind
from string import punctuation
import re

df = pd.read_csv(dataPath + '/ImdbReviews.csv', encoding = 'utf-8')


In [10]:
# 

df[:5]


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [12]:
# Separate the words and count each words occurrence

from collections import Counter

counts = Counter()
pbar = pyprind.ProgBar(len(df['review']), title = 'Counting words occurrences')

for i, review in enumerate(df['review']):
    text = ''.join([c if c not in punctuation else ' ' + c + ' ' for c in review]).lower()
    df.loc[i, 'review'] = text
    pbar.update()
    counts.update(text.split())
    
# Create a mapping of each unique word to an integer
word_counts = sorted(counts, key = counts.get, reverse = True)
print(word_counts[:5])
word_to_int = {word: ii for ii, word in enumerate(word_counts, 1)}

mapped_reviews = []
pbar = pyprind.ProgBar(len(df['review']), title = 'Map review to ints')

for review in df['review']:
    mapped_reviews.append([word_to_int[word] for word in review.split()])
    pbar.update()

Counting words occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:04:36
Map reiew to ints


['the', '.', ',', 'and', 'a']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


This process effectively conerted sequences of words into sequences of integers, but these sequences have different lengths. For this dataset to be ready for an RNN, the sequences need to hae the same length. To accomplish this, we define a paramter called sequence_length that will be set to 200. Sequences that hae fewer than 200 words will be left-padded with zeros, while sequences longer than 200 words will be trimmed so that only the last 200 values will be used. This preprocessing step is implemented in two steps:

1. Create a matrix of zeros, where each row corresponds to a sequence of size 200
2. Fill the index of words in each sequence for the right-hand side of the matrix. If a sequence has a length of only 150, then the first 50 elements of the row would stay zero. It's worth noting that the value chosen for sequence_length is a hyperparameter than can be tuned.

In [14]:
# Create matrix of same-length sequences

sequence_length = 200

sequences = np.zeros((len(mapped_reviews), sequence_length), dtype = int)
for i, row in enumerate(mapped_reviews):
    review_arr = np.array(row)
    sequences[i, -len(row):] = review_arr[-sequence_length:]


In [ ]:
# Create training and test sets
# Note - the reviews were shuffled prior to being saved in a .csv

XTrain = sequences[:25000,:]
yTrain = df.loc[:25000, 'sentiment'].values
XTest = sequences[25000:, :]
yTest = df.loc[25000:, 'sentiment'].values


In [17]:
# Generator helper function for mini-batching

def create_batch_generator(x, y = None, batch_size = 64):
    n_batches = len(x)//batch_size
    x = x[:n_batches * batch_size]
    if y is not None:
        y = y[:n_batches * batch_size]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            yield x[ii:ii + batch_size], y[ii:ii + batch_size]
        else:
            yield x[ii: ii + batch_size]


<a id = 'Embedding'></a>

### Embedding

The data prep stage above created same-length sequences where the elements are integers that correspond to the indices of unique words. Now we need to convert this to input features. The wrong way to do it would be to apply on-ehot encoding to convert indices into ectors of zeros and ones. Each word would be mapped to a vector with a size equal to the number of unique words in the dataset. This is less than ideal, because the number of unique words can rise to the tens of thousands. A model trained on features like this may suffer from the curse of dimensionality. Further, these features would be very sparse, since all values are zero except one.

A better approach would be to map each word to a vector of fixed size with real-valued elements (not integers necessarily). With this approach, we can instead use finite-sized vector to represent and infinite number of real number. This is the idea behind the embedding, which is a feature-learning technique that can be utilized to automatically learn the salient features in the data. Given the value of a parameter unique_words, we can choose the size of the embedding vectors to be much smaller than the number of unique words in the corpus. The advantages of emebedding over one-hot encoding for these problems are:

1. A reduction in dimensionality decreases the effect of the curse of dimensionality
2. The extraction of salient features since the embedding layer in a neural network is trainable

To create an embedding layer, we feed in tf_x as the input layer, which is comprised of vocabulary indices. We create a matrix of size $[n\_words \times embedding\_size]$ as a tensor variable with randomly initialized values between [-1,1]. then we use tf.nn.embedding_lookup to loo up the row in the embedding matrix associated with each element of tf_x:

In [18]:
import tensorflow as tf

In [ ]:
#

embedding = tf.Variable(tf.random_uniform(shape = (n_words, embedding_size), minval = -1, maxval = 1))
embed_x = tf.nn.embedding_lookup(embedding, tf_x)

<a id = 'Building-an-RNN-model'></a>

### Building an RNN model

The SentimentRNN class that we will create has the following methods:

- A constructor to set the model parameters and create a computation graph.
- A build method that declares three placeholder for input data, input labels, and the kee-probability for the dropout process in the hidden layer. It also creates an embedding layer and creates embedded representations as input.
- A train method that creates a session that launches a graph, iterates through mini-batches of data, run for a # of epochs, minimizing the cost along the way before saing the model
- A predict method that creates a new session with the model as of the latest checkpoint saved at the end of the training process, and carries out the predictions on the test data.

In [21]:
# 

class SentimentRNN():
    def __init__(self, n_words, seq_len = 200, lstm_size = 256, num_layers = 1, batch_size = 64
                 , learning_rate = 0.0001, embed_size = 200):
        """
        n_words - must be set equal to the number of unique words (+1, since we use 
                    zero to fill sequences with a size less than 200) and it's used
                    to create the embedding layer, along with embed_size
        embed_size - used with n_words to create the embedding layer
        seq_len- must be set according to the length of the sequences that were created
                    in the preprocessing steps above
        lstm_size - a hyperparameter that determines the number of hidden units in each RNN layer
        """
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size # number of hidden units
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size
        
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build()
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()
    
    def build(self):
        tf_x = tf.placeholder(tf.int32, shape = (self.batch_size, self.seq_len), name = 'tf_x')
        tf_y = tf.placeholder(tf.float32, shape = (self.batch_size), name = 'tf_y')
        tf_keepprob = tf.placeholder(tf.float32, name = 'tf_keepprob')
        
        # create embedding layer
        embedding = tf.Variable(tf.random_uniform((self.n_words, self.embed_size)
                                                  , minval = -1, maxval = 1)
                                        , name = 'embedding')
        embed_x = tf.nn.embedding_lookup(embedding, tf_x, name = 'embeded_x')
        
        # define LSTM cell and stack together
        cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(
                                                tf.contrib.rnn.BasicLSTMCell(self.lstm_size)
                                                ,output_keep_prob = tf_keepprob)
                                            for i in range(self.num_layers)])
        
        # define the initial state
        self.initial_state = cells.zero_state(self.batch_size, tf.float32)
        print('  << initial state >>  ', self.initial_state)
        
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cells, embed_x
                                                           , initial_state = self.initial_state)
        
        # lstm output shape = [batch_size x max_time x cells.output_size]
        print('\n  << lstm_output >>', lstm_outputs)
        print('\n  << final state >>', self.final_state)
        
        logits = tf.layers.dense(inputs = lstm_outputs[:, -1], units = 1, activation = None, name = 'logits')
        logits = tf.squeeze(logits, name = 'logits_squeezed')
        print('\n  << logits    >>', logits)
        
        y_proba = tf.nn.sigmoid(logits, name = 'probabilities')
        predictions = {'probabilities' : y_proba
                       ,'labels' : tf.cast(tf.round(y_proba), tf.int32, name = 'labels')}
        print('\n  << predictions >>', predictions)
        
        # define cost function
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                                labels = tf_y, logits = logits), name = 'cost')
        
        # define optimizer
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name = 'train_op')
        

In the build method, we create three placeholder for the input, output, and dropout keep-probability. Then we add the embedding layer, which builds the embedded representation of the unique words. Next within the build method, we built the RNN network. This was done in three steps.

1. Define multilayer RNN cells
2. Define initial state of these cells
3. Create and RNN specified by the RNN cells in their initial states

These three steps are unpacked in further detail below:

__Step 1: Define multilayer RNN cells__

The first step is to define the multilayer RNN cells, which was accomplished using a TensorFlow wrapper ckass to define the LSTM cells - BasicLSTMCell. These can be stacked together to form a multilayer RNN using the MultiRNNCell wrapper class. The process of stacking RNN cells with a dropout stage has three nested steps. Described from the inside out:

1. Create RNN cells using tf.contrib.rnn.BasicLSTMCell
2. Apply dropout to the RNN cells using tf.contrib.rnn.DropoutWrapper
3. Make a list of such cells according to the desired number of RNN layer and pass this list to tf.contrib.rnn.MultiRNNCell

This process is completed using a list comprehension in the implementation above.

__Step 2: defining the initiatl states for the RNN cells__

In the architecture of LSTM cells, there are three types of inputs - input data $\textbf{x}^t$, activations of hidden units from the previous time step $\textbf{x}^{t-1}$, and the cell state of the previous time step $\textbf{C}^{t-1}$.

In the above implementation $\textbf{x}^t$ is the embedded embed_x data tensor. We also need to specify the previous state of the cells. If we're starting a new input sequence, we initialize the cell state to a zero state, then for each seubsequent time step we need to store the updated state of the cells to use in the following time step. The initial state in the implementation above is set by calling cells.zero_state

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A